In [1]:
import os
import numpy as np
import pandas as pd

import shapely as sp
import shapely.geometry as sgeom
import cartopy.io.shapereader as shpreader

from math import *


In [2]:
import folium
#import branca.colormap as cm
#import json

In [3]:
fname = 'data/townships/myanmar_township_boundaries.shp'

reader = shpreader.Reader(fname)
tsp_geoms = list(reader.geometries())
tsp_records = list(reader.records())


In [4]:
tsps_df = pd.DataFrame([x.attributes for x in tsp_records])

In [5]:
tsps_df.head(20)

,AREA,DT,DT_PCODE,LABEL2,OBJECTID,SELF_ADMIN,ST,ST_2,ST_PCODE,ST_RG,TS,TS_PCODE,T_NAME_M3,T_NAME_WIN
0,5761.296368,Mohnyin,MMR001D002,Hpakant\n169795,250,,Kachin,Kachin State,MMR001,State,Hpakant,MMR001009,ဖားကန့်,zm;uefU
1,2703.611427,Mongmit,MMR015D008,Mongmit\n61072,163,,Shan (North),Shan State (North),MMR015,State,Mongmit,MMR015017,မိုးမိတ်,rdk;rdwf
2,952.439789,Bago,MMR007D001,Waw\n199032,96,,Bago (East),Bago Region (East),MMR007,Region,Waw,MMR007004,ဝေါ,a0g
3,1918.673414,Pyay,MMR008D001,Paukkhaung\n117164,147,,Bago (West),Bago Region (West),MMR008,Region,Paukkhaung,MMR008002,ပေါက်ခေါင်း,aygufacgif;
4,1178.507634,Pyinoolwin,MMR010D002,Mogoke\n191775,263,,Mandalay,Mandalay Region,MMR010,Region,Mogoke,MMR010011,မိုးကုတ်,rdk;ukwf
5,3088.829054,Bhamo,MMR001D003,Shwegu\n84750,167,,Kachin,Kachin State,MMR001,State,Shwegu,MMR001011,ရွှေကူ,a&Tul
6,2022.951971,Taungoo,MMR007D002,Kyaukkyi\n106008,25,,Bago (East),Bago Region (East),MMR007,Region,Kyaukkyi,MMR007011,ကျောက်ကြီး,ausmufBuD;
7,721.604637,Yangon (South),MMR013D003,Thongwa\n158876,300,,Yangon,Yangon Region,MMR013,Region,Thongwa,MMR013025,သုံးခွ,oHk;cG
8,802.136079,Yangon (South),MMR013D003,Kyauktan\n162931,284,,Yangon,Yangon Region,MMR013,Region,Kyauktan,MMR013024,ကျောက်တန်း,ausmufwef;
9,3635.279943,Hpa-An,MMR003D001,Thandaunggyi\n93611,148,,Kayin,Kayin State,MMR003,State,Thandaunggyi,MMR003004,သံတောင်ကြီး,oHawmiffBuD;


In [6]:
def get_index_of_tsp_location(lat,lon,geoms):
    for i, t in enumerate(geoms):
        if t.contains(sgeom.Point(lon, lat)):
            return i
    return -1

In [7]:
# testing a point in Hpakant, should be index 0 in dataframe

get_index_of_tsp_location(25.6787,96.3858,tsp_geoms)

0

In [8]:
# testing a point not in Myanmar, should return -1

get_index_of_tsp_location(0,1,tsp_geoms)


-1

In [9]:
def get_attributes_of_tsp_location(lat,lon,geoms,df):
    try:
        return df.iloc[get_index_of_tsp_location(lat,lon,geoms),:]
    except:
        return ''

In [10]:
# testing a point in Hlaing Tharyar

get_attributes_of_tsp_location(16.8728,96.0685,tsp_geoms,tsps_df)

AREA                       84.2322
DT                  Yangon (North)
DT_PCODE                MMR013D001
LABEL2        Hlaingtharya\n377632
OBJECTID                       179
SELF_ADMIN                        
ST                          Yangon
ST_2                 Yangon Region
ST_PCODE                    MMR013
ST_RG                       Region
TS                    Hlaingtharya
TS_PCODE                 MMR013008
T_NAME_M3               လှိုင်သာယာ
T_NAME_WIN              vSdkifom,m
Name: 130, dtype: object

In [11]:
def viz_point(lat,lon,geoms,df):
    index = get_index_of_tsp_location(25.6787,96.3858,geoms)
    location_result = get_attributes_of_tsp_location(16.8728,96.0685,geoms,df)
    map = folium.Map(location=[lat,lon], zoom_start=12)
    
    html="""
    <strong>Township: </strong>{tsp}<br/>
    <strong>State/Region: </strong>{st}<br/>
    <strong>Pcode: </strong>{pcode}
    """
    table = folium.Html(html.format(
            tsp = location_result['TS'],
            st = location_result['ST'],
            pcode = location_result['TS_PCODE']
        ), script=True)
    popup = folium.Popup(table)
    
    folium.Marker([lat, lon], popup=popup).add_to(map)
    return map

In [12]:
viz_point(16.8728,96.0685,tsp_geoms,tsps_df)

## Read from csv and add columns with township, state and Pcode

In [13]:
df = pd.read_csv('data/Coordinates.csv',sep=';')

In [14]:
df.head(10)

,ID,longitude,latitude,Unnamed: 3
0,1,95.854584,17.497511,NaN
1,2,98.133858,22.036200,NaN
2,3,99.283318,20.381680,NaN
3,4,98.133858,22.036200,NaN
4,5,98.400002,22.483334,NaN
5,6,97.602486,23.714504,NaN
6,7,96.368736,24.767860,NaN
7,8,96.538742,23.649521,NaN
8,9,96.578339,25.703962,NaN
9,10,96.348228,25.620737,NaN


#### Drop extra column from csv file

In [15]:
df.drop(['Unnamed: 3'], axis=1, inplace=True)

#### Apply the get_attributes_of_tsp_location() function to every row in the dataframe and merge it with original dataframe

In [16]:
df = pd.concat([df,
    df.apply(
        lambda row: get_attributes_of_tsp_location(row['latitude'],row['longitude'],tsp_geoms,tsps_df)
            [['ST','TS','TS_PCODE']],axis=1)],
    axis=1)

In [17]:
df.head()

,ID,longitude,latitude,ST,TS,TS_PCODE
0,1,95.854584,17.497511,Yangon,Taikkyi,MMR013005
1,2,98.133858,22.036200,Shan (South),Kyethi,MMR014015
2,3,99.283318,20.381680,Shan (East),Monghsat,MMR016006
3,4,98.133858,22.036200,Shan (South),Kyethi,MMR014015
4,5,98.400002,22.483334,Shan (North),Tangyan,MMR015004


In [18]:
df.groupby(['ST']).count()

,ID,longitude,latitude,TS,TS_PCODE
ST,,,,,
Ayeyarwady,1,1,1,1,1
Bago (East),6,6,6,6,6
Bago (West),1,1,1,1,1
Chin,9,9,9,9,9
Kachin,199,199,199,199,199
Kayin,46,46,46,46,46
Magway,1,1,1,1,1
Mandalay,9,9,9,9,9
Mon,3,3,3,3,3


#### Save output as a csv file

In [19]:
df.to_csv('data/Coordinates_with_location.csv',sep=',')